In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [31]:
train = pd.read_csv('./data/Train_Fyxd0t8.csv', index_col='ID')
test = pd.read_csv('./data/Test_C1XBIYq.csv', index_col='ID')

In [32]:
train.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
ID,,,,,,,,,
F00000001,188,1,0,1,0,0,0,1,0
F00000003,209,1,0,1,0,0,0,2,1
F00000004,257,1,0,1,0,0,0,2,1
F00000005,257,1,1,1,0,0,0,2,1
F00000006,342,1,0,1,0,0,0,2,1


In [33]:
train.Crop_Damage.value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [34]:
train = train.fillna(-1)
test = test.fillna(-1)

In [35]:
train = train.select_dtypes(exclude=['object'])
test = test.select_dtypes(exclude=['object'])

In [36]:
X = train[train.columns.drop('Crop_Damage')]
y = train.Crop_Damage

In [37]:
from sklearn.cross_validation import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [80]:
scaler = MinMaxScaler()
# clf = LogisticRegression()
# knn = KNeighborsClassifier()
clf = RandomForestClassifier()

pipeline = Pipeline([('scaler', scaler), ('clf', clf)])

params_grid = {'clf__n_estimators': [100, 250, 500], 'clf__min_samples_split': [3, 5, 7]}

grid = GridSearchCV(pipeline, params_grid, cv=5, scoring='accuracy')

In [81]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'clf__min_samples_split': [3, 5, 7], 'clf__n_estimators': [100, 250, 500]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [82]:
print grid.best_params_

{'clf__n_estimators': 500, 'clf__min_samples_split': 7}


In [83]:
print grid.best_score_

0.841093886279


In [69]:
from sklearn.metrics import classification_report, confusion_matrix

In [84]:
predsTrain = grid.best_estimator_.predict(X_train)
predsTest = grid.best_estimator_.predict(X_test)

In [85]:
print 'Accuracy score on the training examples ', classification_report(y_train, predsTrain)
print 'Accuracy score on the test examples ', classification_report(y_test, predsTest)

Accuracy score on the training examples               precision    recall  f1-score   support

          0       0.90      0.99      0.94     59401
          1       0.84      0.43      0.57      9850
          2       0.85      0.13      0.22      1835

avg / total       0.89      0.89      0.87     71086

Accuracy score on the test examples               precision    recall  f1-score   support

          0       0.86      0.97      0.91     14837
          1       0.47      0.19      0.27      2457
          2       0.17      0.02      0.03       478

avg / total       0.79      0.84      0.80     17772



In [72]:
# fit on the whole dataset
grid.best_estimator_.fit(X, y)

Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=10, p=2, weights='uniform'))])

In [ ]:
predictions = grid.best_estimator_.predict(test)

In [74]:
submissions = pd.read_csv('./data/Sample_Submission_Psj3sjG.csv')

In [75]:
submissions['ID'] = test.index.values
submissions['Crop_Damage'] = predictions

In [76]:
submissions.to_csv('./submissions/fourth.csv', index=False)